<a href="https://colab.research.google.com/github/MateoProjects/UtilsAI/blob/main/Tensorflow_Guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow Simple Guide

Tensorflow in 10 minuts. A simple guide with util functions that can be used.

**IN PROGRES** 

## Imports

In [ ]:
import tensorflow as tf
import tensorflow.keras.applications as app
import keras
import matplotlib.pyplot as plt
import numpy as np
from keras import layers, regularizers
from keras import backend as K
from tensorflow.keras import optimizers
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,  GlobalAveragePooling2D
from tensorflow.keras.utils import Sequence
from tensorflow.keras import utils
from tensorflow.keras.models import Model
from skimage import io
from keras.utils.data_utils import Sequence
from copy import copy
from constants import *


## Layers

Keras have a convolutonal and dense layers.

**Note**: The cells below are only examples of functions. Don't execute. 

In [ ]:
model = keras.Sequentia() # this line is for generate a red in sequential

If we want to add new layers we only need to use the function **model.add()**

In case that we want to add a new dense layer:
* 

```Python
tf.keras.layers.Dense(
    units,
    activation=None,
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    **kwargs
)
```



Where: 
* Units activation is a integer.
* Activation is a name of activation function like "relu", "gelu" etc...
* kerne_initializer can be a string or a Initializer. See : https://keras.io/api/layers/initializers/

In [ ]:
model.add(layers.Dense(128, activation='relu'))

In case that we want to add a new conv layer:

* 

```Python
tf.keras.layers.Conv2D(
    filters,
    kernel_size,
    strides=(1, 1),
    padding='valid',
    data_format=None,
    dilation_rate=(1, 1),
    groups=1,
    activation=None,
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    **kwargs
)
```



In [ ]:
model.add(layers.Conv2D(16,kernel_size(3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
#If its the first layers then:
model.add(layers.Conv2D(16,kernel_size(3,3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(256,256,3), data_format='channels_last'))



After a convolutional layer we need to add a pooling layer. In this example we show how to add an AveragePooling layer and MaxPooling layer.

```
tf.keras.layers.AveragePooling2D(
    pool_size=(2, 2), strides=None, padding="valid", data_format=None, **kwargs
)
```

or 

```
tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2), strides=None, padding="valid", data_format=None, **kwargs
)
```

Link to the documentation: https://keras.io/api/layers/pooling_layers/

In [ ]:
model.add(layers.Conv2D(16,kernel_size(3,3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(256,256,3), data_format='channels_last'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# or 

model.add(layers.Conv2D(16,kernel_size(3,3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(256,256,3), data_format='channels_last'))
model.add(layers.AveragePooling2D(pool_size=(2,2)))




If we want to normalize the output of a layer we can add a BatchNormalization layer.

**Link to the documentation**: https://keras.io/api/layers/normalization_layers/batch_normalization/

In [ ]:
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='gelu', kernel_initializer='he_uniform',padding='same'))
model.add(layers.BatchNormalization(center=True,))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

## Exemple of code for Museum Dataset

The MAMe dataset contains images of high-resolution and variable shape of artworks from 3 different museums:

* The Metropolitan Museum of Art of New York
* The Los Angeles County Museum of Art
* The Cleveland Museum of Art

In [ ]:
def get_baseModel():
    """
    Get the model from the base model.
    @return model
    """

    model = keras.Sequential()
    model.add(layers.Conv2D(40, kernel_size=(3, 3), activation='gelu', kernel_initializer='he_uniform', padding='same', input_shape=(256,256,3), data_format='channels_last'))
    model.add(layers.BatchNormalization(center=True,))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(80, kernel_size=(3, 3), activation='gelu', kernel_initializer='he_uniform', padding='same'))
    model.add(layers.Dropout(0.2))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(160, kernel_size=(3, 3), activation='gelu', kernel_initializer='he_uniform',padding='same'))
    model.add(layers.BatchNormalization(center=True,))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())
    #model.add(layers.Dense(256, activation='gelu'))
    #model.add(layers.Dropout(0.2))
    model.add(layers.Dense(128, activation='gelu' ))
    model.add(layers.Dropout(0.2))
    model.add(layers.BatchNormalization(center=True,))
    model.add(layers.Dense(NUM_CLASSES, activation='softmax'))
    model.summary()
    return model

With this model you can obtain a 68% of accuracy in validation without data Augmentation. 

## Training a model

Once we have our architecture of the model we need decide if we want a optimizer
 
The most tipical optimizers are: 

* SGD:


```
tf.keras.optimizers.SGD(learning_rate=0.1)
```
* Adam:

```
tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam",
    **kwargs
)
```

**Link to documentation**: https://keras.io/api/optimizers/






In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
# or
optimizer = keras.optimizers.SGD(learning_rate=0.001)

Once we have chosed the optimizer we need to compile the model that configures the model for training. We can fix the loss function that we want and the metrics that we want to evaluate. 

```
Model.compile(
    optimizer="rmsprop",
    loss=None,
    metrics=None,
    loss_weights=None,
    weighted_metrics=None,
    run_eagerly=None,
    steps_per_execution=None,
    jit_compile=None,
    **kwargs
)

```

**Note**: Loss can be a string or a [tf.losses](https://www.tensorflow.org/api_docs/python/tf/keras/losses)

**Link to documentation**: https://keras.io/api/models/model_training_apis/#compile-method

In [ ]:
model = get_baseModel()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


## Train 

For train the model we need to call model.fit()

```
Model.fit(
    x=None,
    y=None,
    batch_size=None,
    epochs=1,
    verbose="auto",
    callbacks=None,
    validation_split=0.0,
    validation_data=None,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)
```

**Notes**:

* X and Y can be a generators, numpy arrays etc... If we use a DataGenerators we don't need to specify Y

**Link to documentation**: https://keras.io/api/models/model_training_apis/#fit-method

In [ ]:
mdl_fit = model.fit(x_train, y_train, validation_data=validationData, 
                    shuffle=True,epochs=EPOCHS, batch_size=BATCH_SIZE, 
                    use_multiprocessing=True,workers=4,callbacks = [early])

### **IMPORTANT THING**

By default Keras' model.fit() returns a History callback object. This object keeps track of the accuracy, loss and other training metrics, for each epoch, in the memory.

This object is a dictionary where keys are the metrics.

**Example**:
```
print(mdl_fit['accuracy'])
print(mdl_fit['val_loss'])
```

**Note**: This is only an example. Don't execute it. It's only for see how to call the function.

**Other parameters**:
* Shuffle will shuffle the data during the training process. 
* In this example validation data is passed using a Data generator. 
* If we wan to add a early stopping we need to put in to callbacks. Callbacks are explained below. 

### Adding a Early Stopping

If we want to add a early stopping we need to add a new callback.

**Link to documentation**: https://keras.io/api/callbacks/

In [ ]:
early = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=1, mode='auto', restore_best_weights=True)

In the parameters of this function we can see how we can decide the variable that we want to monitor.
* The patience it's for indicate the number of epochs with no improvement after which training will be stopped.
* Verbose 0 if we dont want to see anything else 1. 

## Evaluating a model

If we used a validation set during the training one form to see the accuracy and loss during the training is plot these using matplotlib. 

The function below plot loss and accuracy. 

In [ ]:
def print_results_model(mdl_fit, epochs=16, batch_size=32):
    """
    Print the results of the model trained on the given data.
    @param mdl_fit: model trained on the given data
    @param epochs: number of epochs used for training
    @param batch_size: batch size used for training
    @param balanced: whether the data is balanced or not
    """
    # plot the loss
    plt.plot(mdl_fit.history['loss'], label='train loss')
    plt.plot(mdl_fit.history['val_loss'], label='val loss')
    plt.legend()
    plt.savefig("data_epochs_" +str(epochs)+"_batchSize_"+ str(batch_size)+'_LossVal_loss')
    plt.show()
    # plot the accuracy
    plt.plot(mdl_fit.history['accuracy'], label='train acc')
    plt.plot(mdl_fit.history['val_accuracy'], label='val acc')
    plt.legend()
    plt.savefig("balanced_data_epochs_" +str(epochs)+"_batchSize_"+ str(batch_size)+'_AccVal_acc')
    #save model to disk
    plt.show()